In [1]:
import numpy as np
import os

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras.backend as K

## Implementation

In [ ]:
class AlexNetWhole:
    def build(self, output_dim):
        self.output_dim = output_dim
        
        model = keras.models.Sequential([
            keras.layers.Conv2D(filters=96, 
                            kernel_size=11, 
                            strides=4,
                            padding="valid", 
                            activation="relu", 
                            input_shape=[227, 227, 3]),
            
            keras.layers.Lambda(tf.nn.local_response_normalization),
            
            keras.layers.MaxPool2D(),
            
            keras.layers.Conv2D(filters=256, 
                          kernel_size=5, 
                          strides=1,
                          padding="same", 
                          activation="relu"),
            
            keras.layers.Lambda(tf.nn.local_response_normalization),
            
            keras.layers.MaxPool2D(),
            
            keras.layers.Conv2D(filters=384, 
                          kernel_size=5, 
                          strides=1,
                          padding="same", 
                          activation="relu"),
            keras.layers.Conv2D(filters=384, 
                          kernel_size=5, 
                          strides=1,
                          padding="same", 
                          activation="relu"),
            keras.layers.Conv2D(filters=256, 
                          kernel_size=5, 
                          strides=1,
                          padding="same", 
                          activation="relu"),
            keras.layers.MaxPool2D(),
            
            keras.layers.Flatten(),
            keras.layers.Dense(4096, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(4096, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(self.output_dim, activation="softmax")
        ])
        return model

## Part by Part Implementation

In [ ]:
class AlexNetBlock1(keras.layers.Layer):
    def __init__(self, channels=3, **kwargs):
        super().__init__(**kwargs)
        self.channels = channels
        self.block_layers = [
            keras.layers.Conv2D(filters=96, 
                            kernel_size=11, 
                            strides=4,
                            padding="valid", 
                            activation="relu", 
                            input_shape=[227, 227, 3]),
            
           keras.layers.Lambda(tf.nn.local_response_normalization),
        
            keras.layers.MaxPool2D(),
            
            keras.layers.Conv2D(filters=256, 
                          kernel_size=5, 
                          strides=1,
                          padding="same", 
                          activation="relu"),
           keras.layers.Lambda(tf.nn.local_response_normalization),
        ]
        
        self.out_l = keras.layers.MaxPool2D()
    def call(self, inputs):
        Z = inputs
        for layer in self.block_layers:
            Z = layer(Z)
        return self.out_l(Z)

In [ ]:
class AlexNetBlock2(keras.layers.Layer):
    def __init__(self, n_layers=3, **kwargs):
        super().__init__(**kwargs)
        self.block_layers = [
            keras.layers.Conv2D(filters=384, 
                            kernel_size=3, 
                            strides=1,
                            padding="same", 
                            activation="relu") for _ in range(2)
        ]
        
        self.end_layers = [
            keras.layers.Conv2D(filters=256, 
                            kernel_size=3, 
                            strides=1,
                            padding="same", 
                            activation="relu"),
        ]
        self.out_l2 = keras.layers.MaxPool2D()
        
    def call(self, inputs):
        Z = inputs
        for layer1 in self.block_layers:
            Z = layer1(Z)
        for layer2 in self.end_layers:
            Z = layer2(Z)
        return self.out_l2(Z)

In [ ]:
class AlexNet(keras.models.Model):
    def __init__(self, output=3, channels=3, **kwargs):
        super().__init__(**kwargs)
        self.channels = channels
        self.my_output = output
        self.block1 = AlexNetBlock1(1)
        self.block2 = AlexNetBlock2(3)
        
        self.flatt = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(4096, activation="relu")
        self.drop1 = keras.layers.Dropout(0.5)
        self.fc2 = keras.layers.Dense(4096, activation="relu")
        self.drop2 = keras.layers.Dropout(0.5)
        self.out_fc = keras.layers.Dense(self.my_output, activation="softmax")
    
    def call(self, inputs):
        Z = self.block1(inputs)
        Z = self.block2(Z)
        Z = self.flatt(Z)
        Z = self.fc1(Z)
        Z = self.drop1(Z)
        Z = self.fc2(Z)
        Z = self.drop2(Z)
        Z = self.out_fc(Z)
        return Z